In [ ]:
import numpy as np
import rioxarray as rxr
import rasterio
import xarray as xr
from tools import load_ana_data
from glob import glob
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from tools import lonlim, latlim, process_satellite_image, standardize_lat_lon, rasterize_geodataframe
from shapely.geometry import box, Point, shape
import geopandas as gpd
import fiona
fiona.drvsupport.supported_drivers['libkml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw' # enable KML support which is disabled by default

In [ ]:
fnames = glob("../data/external/hydroweb/5-*")

ana = []
station = []
for fname in tqdm(fnames):
    ana.append(load_ana_data(fname))
    station.append(fname.split("-")[-4])
ana = xr.concat(ana, "station").assign_coords(station=station)


In [ ]:
# Your longitude and latitude
longitude = ana.longitude.values
latitude = ana.latitude.values

# Create a DataFrame for your point
df = pd.DataFrame({"longitude": longitude, "latitude": latitude})

In [ ]:
dx = dy = 30
dlon = dx/(111.2e3*np.cos(latitude.mean()*np.pi/180))
dlat = dy/(111.2e3)

fnames = glob(f"../data/external/swot/*")
fnames.sort()
fnames = np.array(fnames)

variables = [
    "height", "water_frac", "classification",
    "geoid", "illumination_time", "pixel_area"
]

heights = []
for fname in tqdm(fnames):
    swot = xr.open_dataset(fname, group="pixel_cloud")[variables]
    swot.load()
    
    for lon, lat, station in zip(ana.longitude.values, ana.latitude.values, ana.station.values):

        where = (
            (swot.longitude>lon-dlon)&(swot.longitude<lon+dlon)&
            (swot.latitude>lat-dlat)&(swot.latitude<lat+dlat)
        ).values
        
        ind = np.argwhere(where).ravel()

        where = (
            (swot.classification>2)&(swot.classification<6)&
            (swot.water_frac>0.1)
        )
        dsi = swot.where(where).sel(points=ind)

        dsi["water_level"] = (dsi.height-dsi.geoid)
        dsi = dsi.dropna("points")
        
        if dsi.points.size>1:

            level = xr.merge([
                dsi.illumination_time.mean(),
                dsi["water_level"].median(),
            ])

            level["time"] = level.illumination_time.mean()
            level = level.set_coords("time").drop_vars("illumination_time").expand_dims("time")

            level = level.assign_coords(station=station).expand_dims("station")
            
            heights.append(level)

ana_swot = xr.merge(heights)
ana_swot = ana_swot.isel(time=np.argsort(ana_swot.time.values))
ana_swot = ana_swot.dropna("time", how="all")
ana_swot["time"] = ana_swot["time"] - np.timedelta64(3, "h")
ana_swot["time"].attrs["timezone"] = "UTC-3"

correction = (ana.height.interp(time=ana_swot.time)-ana_swot.water_level).median()
ana_swot["water_level"] += correction
ana_swot.attrs["correction (m)"] = correction.values
ana_swot["water_level"].attrs = {"units": "m", "long_name": "height above geoid"}

ana_swot.to_netcdf("../data/processed/swot_ana.nc")
ana.to_netcdf("../data/processed/ana.nc")

In [ ]:
colors = ["#e66231ff", "#1f59ffff"]
style = dict(marker="*", lw=0, markersize=8, markeredgewidth=0.6, zorder=10, markeredgecolor="0.2")

before_i, after_i = 14, 15

fig, ax = plt.subplots(figsize=(8,4))

for time in ana_swot.time.values[[before_i, after_i]]:
    ax.axvline(time, ls="--", color="0.3", alpha=0.6)
    
for station, color in zip(ana.station, colors):
    ana_swot.water_level.sel(station=station).plot(ax=ax, x="time", color=color, **style)
    ana.sel(station=station).height.plot(ax=ax, x="time", color=color, label=station.values)
ax.grid(True, ls="--", alpha=0.5)
ax.legend()
ax.set(title="", xlabel="", ylabel="water level [m]")


In [ ]:
fnames = [
    "../data/external/landsat/before.nc",
    "../data/external/landsat/after.nc"
]
landsat = []
for fname in fnames:
    landsat.append(xr.open_dataset(fname).load())

In [ ]:
before_img = process_satellite_image(landsat[0])
after_img = process_satellite_image(landsat[1])

before_img = before_img.sel(longitude=slice(*lonlim), latitude=slice(*latlim))
after_img = after_img.sel(longitude=slice(*lonlim), latitude=slice(*latlim))

In [ ]:
da = landsat[0]["SCL"]

mask = (da==6).astype('uint8').T
mask = mask.rolling(lon=11, lat=11, center=True, min_periods=1).max()

In [ ]:
# Get resolution from the coordinate differences
xres = (da.lon[1] - da.lon[0]).item()  # Convert to native Python float
yres = (da.lat[0] - da.lat[1]).item()

# Create the transform
transform = rasterio.transform.from_origin(
    da.lon[0], da.lat[0], xres, yres
)

# Convert the mask to shapes using rasterio.features.shapes
shapes = list(rasterio.features.shapes(mask, transform=transform))

# Filter out empty geometries (if any)
valid_shapes = [(shape(geom), val) for geom, val in shapes if val == 1]  # val == 1 means the mask is True

# Create a GeoDataFrame with the shapes and assign a CRS (if your raster has one)
gdf = gpd.GeoDataFrame(geometry=[shape[0] for shape in valid_shapes], crs=da.rio.crs)

In [ ]:
mask = (landsat[1]["SCL"]==6).astype('uint8').T

# Convert the mask to shapes using rasterio.features.shapes
shapes = list(rasterio.features.shapes(mask, transform=transform))

# Filter out empty geometries (if any)
valid_shapes = [(shape(geom), val) for geom, val in shapes if val == 1]  # val == 1 means the mask is True

# Create a GeoDataFrame with the shapes and assign a CRS (if your raster has one)
scl_after = gpd.GeoDataFrame(geometry=[shape[0] for shape in valid_shapes], crs=da.rio.crs)

In [ ]:
def count_vertices(geom):
    if geom.geom_type == 'Polygon':
        return len(geom.exterior.coords)
    elif geom.geom_type == 'MultiPolygon':
        return sum(len(poly.exterior.coords) for poly in geom)
    else:
        return None  # Handle non-polygon geometries if present

gdf["count"] = gdf.geometry.apply(count_vertices)
gdf = gdf[gdf["count"]>400]

In [ ]:
flooded_areas = gpd.read_file("../data/external/ufrgs/inundacao_em_6_de_maio_de_2024.kml")[["geometry"]]
crs = flooded_areas.crs
flooded_areas["land"] = True
gdf["land"] = False
flooded_areas = pd.concat([flooded_areas.to_crs(4326), gdf[["geometry", "land"]].to_crs(4326)]).reset_index(drop=True)

In [ ]:
bbox = box(lonlim[0], latlim[0], lonlim[1], latlim[1])
before_gdf = gpd.GeoDataFrame(geometry=[flooded_areas[flooded_areas.land==False].unary_union])
after_gdf = gpd.GeoDataFrame(geometry=[flooded_areas.unary_union])

before_gdf.crs=crs
after_gdf.crs=crs

before_gdf = gpd.clip(before_gdf, bbox)
after_gdf = gpd.clip(after_gdf, bbox)

before_raster = rasterize_geodataframe(before_gdf, lonlim, latlim, resolution=0.001)
after_raster = rasterize_geodataframe(after_gdf, lonlim, latlim, resolution=0.001)

In [ ]:
elevation = standardize_lat_lon(xr.open_dataset("../data/external/landsat/elevation.nc").elevation)
elevation = elevation.sel(longitude=slice(*lonlim), latitude=slice(*latlim))

In [ ]:
gdf = gpd.read_file("../data/external/shapefiles/RM_Porto_Alegre/RM_PortoAlegre_UDH.shp")
gdf = gdf.rename({"UDH_ATLAS": "udh"}, axis=1)
gdf["udh"] = gdf["udh"].astype("int")
gdf = gdf.set_index("udh")

fname = "../data/external/shapefiles/RM_Porto_Alegre/atlasivs_dadosbrutos_Porto_Alegre.xlsx"
df = pd.read_excel(fname, sheet_name="UDH")
df = df[df.ano==2010].reset_index(drop=True).set_index("udh")
gdf = gdf.join(df.loc[gdf.index])

In [ ]:
intersection = gdf.intersection(bbox)
perc_area = (intersection.to_crs(32722).area/gdf.to_crs(32722).area)
gdf = gdf[perc_area>0.5]

In [ ]:
gdf.hvplot(geo=True, color="ivs")*before_gdf.hvplot(geo=True)*after_gdf.hvplot(geo=True, alpha=0.4)

In [ ]:
area = gdf.to_crs(32722).intersection(after_gdf.to_crs(32722).geometry.values[0]).area
intersection = gdf.copy()
intersection["intersection_area"] = area
intersection["area"] = gdf.to_crs(32722).area
intersection = intersection[intersection.intersection_area>0]

In [ ]:
((intersection["populacao"]*(intersection["t_vulner"]/100)*(intersection["intersection_area"]/intersection["area"]))).sum()

In [ ]:
(intersection["populacao"]*(intersection["intersection_area"]/intersection["area"])).sum()

In [ ]:
gdf.prosp_soc.groupby(gdf.prosp_soc).count()/gdf.index.size

In [ ]:
intersection.prosp_soc.groupby(intersection.prosp_soc).count()/intersection.index.size

In [ ]:
(intersection.ivs*intersection.populacao).sum()/intersection.populacao.sum()

In [ ]:
(gdf.ivs*gdf.populacao).sum()/gdf.populacao.sum()

In [ ]:
(intersection.t_vulner*intersection.populacao/100).sum()

In [ ]:
(gdf.t_vulner*1e-2*gdf.populacao).sum()/gdf.populacao.sum()

In [ ]:
(intersection.t_vulner*1e-2*intersection.populacao).sum()/intersection.populacao.sum()

In [ ]:
gdf.ivs.plot.hist(alpha=0.3, density=True, bins=np.arange(0, 0.5, 0.02))
intersection.ivs.plot.hist(alpha=0.3, density=True, bins=np.arange(0, 0.5, 0.02))

In [ ]:
fnames = glob(f"../data/external/swot/*")
fnames.sort()
fnames = np.array(fnames)

swot = []

for i, (fname, raster) in enumerate(zip(fnames[[9, 11]], (before_raster, after_raster))):
    swoti = standardize_lat_lon(xr.open_dataset(fname, group="pixel_cloud")[variables])
    swoti.load()

    water = raster.interp(longitude=swoti.longitude, latitude=swoti.latitude, method="nearest")
    
    where = (
        (swoti.classification>2)&(swoti.classification<6)&
        (swoti.water_frac>0.1)&(water==1)
    )
    
    ind = np.argwhere(where.values).ravel()
    
    swoti = swoti.isel(points=ind)
    
    water_level = (swoti.height-swoti.geoid+correction).rename("water_level")

    swoti = swoti.where(water_level>0, drop=True)

    water_level = water_level.where(water_level>0, drop=True)

    elevation_swot = elevation.interp(longitude=water_level.longitude, latitude=water_level.latitude, method="nearest").rename("elevation_srtm")
    water_depth = (water_level-elevation_swot).rename("water_depth")
    water_volume = (water_depth*swoti.pixel_area).rename("water_volume")

    swoti = xr.merge([swoti, water_depth, water_level, water_volume, elevation_swot])
    
    swot.append(swoti)

# Landcover

In [ ]:
colormap_dict = {
    10: '#006400',  # Tree cover
    20: '#ffbb22',  # Shrubland
    30: '#ffff4c',  # Grassland
    40: '#f096ff',  # Cropland
    50: '#fa0000',  # Built-up
    60: '#b4b4b4',  # Bare / sparse vegetation
    70: '#f0f0f0',  # Snow and ice
    80: '#0064c8',  # Permanent water bodies
    90: '#0096a0',  # Herbaceous wetland
    95: '#00cf75',  # Mangroves
    100: '#fae6a0', # Moss and lichen
}
clim = (2.5, 102.5)
cmap = xr.DataArray([colormap_dict[i] for i in list(colormap_dict)], dims=["value"], coords=dict(value=list(colormap_dict)))
cmap = cmap.sel(value=np.arange(*clim,2.5), method="nearest").assign_coords(value=np.arange(*clim,2.5))

In [ ]:
landcover = standardize_lat_lon(xr.open_dataset("../data/external/landsat/landcover.nc")).Map

In [ ]:
landcover_plot = landcover.hvplot(
        x='longitude', 
        y='latitude',
        geo=True,
        rasterize=True, 
        clim=clim,
        cmap=cmap.values.tolist(),
        clabel="Land Cover Type",
        aggregator="mean",
        colorbar=False
    )#.opts(cticks=list(colormap_dict))

# Plots

In [ ]:
xlim = (lonlim[0], lonlim[1])
ylim = (latlim[0], latlim[1])

kw = dict(
    water_depth = dict(x="longitude", y="latitude", color="water_depth", colorbar=False, geo=True, rasterize=True, clim=(0,9), x_sampling=0.0002, y_sampling=0.0002),
    water_level = dict(x="longitude", y="latitude", color="water_level", colorbar=False, geo=True, rasterize=True, clim=(0,7), x_sampling=0.0002, y_sampling=0.0002),
    geoid = dict(x="longitude", y="latitude", color="geoid", colorbar=False, geo=True, rasterize=True, clim=(4,6), x_sampling=0.0002, y_sampling=0.0002),
    elevation = dict(geo=True, x="longitude", y="latitude", color="elevation", rasterize=True, colorbar=False),
    stations = dict(x="longitude", y="latitude", geo=True, color="red"),
    area = dict(geo=True, line_width=1, color=None),
    rgb = dict(x="longitude", y="latitude", bands="bands", geo=True, rasterize=True),
)
for k in kw:
    kw[k] = dict(height=350, width=350, **kw[k])
    # kw[k] = dict(xlim=xlim, ylim=ylim, **kw[k])

In [ ]:
(
    swot[0].water_level.hvplot.points(**kw["water_level"])+
    swot[1].water_level.hvplot.points(**kw["water_level"])+
    swot[1].geoid.hvplot.points(**kw["geoid"])+
    np.log10(elevation).where(elevation>0,0).hvplot(**kw["elevation"])
).cols(2)

In [ ]:
before_plot = (
    (
        before_img.hvplot.rgb(**kw["rgb"])
        *before_gdf.hvplot(**kw["area"], line_color="blue")
        *after_gdf.hvplot(**kw["area"], line_color="red")
    ).opts(title="a) Apr 15th 2024 (Landsat)")+
    (
        landcover_plot*swot[0].water_depth.hvplot.points(**kw["water_depth"])*
        before_gdf.hvplot(**kw["area"], line_color="blue")*
        after_gdf.hvplot(**kw["area"], line_color="black")
    ).opts(title="b) water depth and land cover")
).cols(1)

after_plot = (
    (
        after_img.hvplot.rgb(**kw["rgb"])*
        before_gdf.hvplot(**kw["area"], line_color="blue")*
        after_gdf.hvplot(**kw["area"], line_color="red")
    ).opts(title="c) May 6th 2024 (Landsat)")+
    (
        swot[1].water_depth.hvplot.points(**kw["water_depth"])*
        before_gdf.hvplot(**kw["area"], line_color="blue")*
        after_gdf.hvplot(**kw["area"], line_color="red")
    ).opts(title="d) water depth")
).cols(1)

(before_plot+after_plot).cols(2)

In [ ]:
swot[1].water_depth.hvplot.points(x="longitude", y="latitude", color="water_depth", geo=True, rasterize=True, clim=(0,9), x_sampling=0.0002, y_sampling=0.0002)

In [ ]:
kw = dict(
    water_depth = dict(x="longitude", y="latitude", color="water_depth", geo=True, rasterize=True, clim=(0,9), x_sampling=0.0002, y_sampling=0.0002),
    water_level = dict(x="longitude", y="latitude", color="water_level", geo=True, rasterize=True, clim=(0,7), x_sampling=0.0002, y_sampling=0.0002),
    geoid = dict(x="longitude", y="latitude", color="geoid", geo=True, rasterize=True, clim=(4,6), x_sampling=0.0002, y_sampling=0.0002),
    elevation = dict(geo=True, x="longitude", y="latitude", color="elevation", rasterize=True),
    stations = dict(x="longitude", y="latitude", geo=True, color="red"),
    area = dict(geo=True, line_width=1, color=None),
    rgb = dict(x="longitude", y="latitude", bands="bands", geo=True, rasterize=True),
)
for k in kw:
    kw[k] = dict(height=350, width=350, **kw[k])
    # kw[k] = dict(xlim=xlim, ylim=ylim, **kw[k])

(
    swot[0].water_level.hvplot.points(**kw["water_level"]).opts(title="Apr 15th 2024")+
    swot[1].water_level.hvplot.points(**kw["water_level"]).opts(title="May 6th 2024")+
    swot[1].geoid.hvplot.points(**kw["geoid"]).opts(title="Geoid")+
    np.log10(elevation).where(elevation>0,0).hvplot(**kw["elevation"]).opts(title="SRTM elevation")
).cols(2)

In [ ]:
swot[1].water_depth.hvplot.hist(bins=np.arange(-10,10,0.5))

In [ ]:
mask = after_raster.interp(longitude=landcover.longitude, latitude=landcover.latitude)==1

In [ ]:
landcover.where(mask).plot.hist(bins=cmap.value.values);

In [ ]:
total_area = (before_raster.size*(resolution*111.2e3)**2)
npoints = before_raster.size

In [ ]:
area_before = total_area*before_raster.sum().values/npoints
area_after = total_area*after_raster.sum().values/npoints

In [ ]:
(area_after-area_before)

In [ ]:
water_volume.sum().values/1000

In [ ]:
mi_l = (((raster.size*(resolution*111.2e3)**2)/area.sum().values)*water_volume.sum().values/1000)*1e-6
print(f"{mi_l:0.2f} millions liters")

In [ ]:
columbia = (7500/1000)*1e-6
(mi_l/columbia)/86400

In [ ]:
((raster.size*(resolution*111.2e3)**2)/area.sum().values)

In [ ]:
ds = swot[1]

dx = dy = 10000
dlon = dx/(111.2e3*np.cos(latitudes.mean()*np.pi/180))
dlat = dy/(111.2e3)

n = 1000
ind = np.random.randint(0, ds.points.size-1, n)
longitudes, latitudes, water_levels = ds.longitude.values, ds.latitude.values, ds.water_level.values
for lon, lat, water_level in tqdm(zip(longitudes[ind], latitudes[ind], water_levels[ind]), total=n):

    where = (
        (longitudes>lon-dlon)&(longitudes<lon+dlon)&
        (latitudes>lat-dlat)&(latitudes<lat+dlat)
    )
    
    i = np.argwhere(where).ravel()[np.random.randint(0, where.sum()-1, 5*n)]


In [ ]:
(29.132/44.5, 23.9/44.5)

In [ ]:
distance = np.sqrt((lon-longitudes[i])**2+(lat-latitudes[i])**2)
correlation = (water_levels[i]-water_level)**2

In [ ]:
import numpy as np
import scipy.spatial.distance as ssd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Function to model the Gaussian correlation function
def gaussian_correlation(r, lc, epsilon):
    return (1 - epsilon**2) * np.exp(-r**2 / lc**2)

# Function to compute pairwise distances and correlations
def compute_lengthscale(x, y, z, epsilon):
    # Create an array of the coordinates
    coords = np.column_stack((x, y))
    
    # Compute the pairwise distances
    dists = ssd.pdist(coords)
    dists_matrix = ssd.squareform(dists)
    
    # Compute the pairwise correlations
    z_diffs = ssd.pdist(z[:, None])
    correlations = np.exp(-z_diffs / z_diffs.std())
    
    # Fit the Gaussian correlation model to the data
    params, _ = curve_fit(lambda r, lc: gaussian_correlation(r, lc, epsilon), dists, correlations)
    lengthscale = params[0]
    
    return lengthscale, dists, correlations, params

# Function to compute the gridded field using Objective Analysis (OA)
def objective_analysis(x, y, z, lc, epsilon):
    # Create an array of the coordinates
    coords = np.column_stack((x, y))
    
    # Compute the pairwise distances
    dists = ssd.pdist(coords)
    dists_matrix = ssd.squareform(dists)
    
    # Compute the correlation matrix
    C = gaussian_correlation(dists_matrix, lc, epsilon)
    
    # Compute the gridded field using the weighted observations
    theta = np.dot(C, z) / C.sum(axis=1)
    
    return theta, dists_matrix, C

# Generate synthetic data with a known correlation lengthscale
def generate_synthetic_data(n_points, true_lengthscale, noise_level):
    np.random.seed(0)
    x = np.random.rand(n_points)
    y = np.random.rand(n_points)
    coords = np.column_stack((x, y))
    
    # Compute distances
    dists = ssd.pdist(coords)
    dists_matrix = ssd.squareform(dists)
    
    # Generate correlated z values with Gaussian correlation
    z = np.exp(-dists_matrix / true_lengthscale).sum(axis=1)
    
    # Add noise
    z += noise_level * np.random.randn(n_points)
    
    return x, y, z

In [ ]:
# Parameters
n_points = 2000
true_lengthscale = 0.1
noise_level = 0.5
epsilon = 0.116  # Given in the image

# Generate synthetic data
x, y, z = generate_synthetic_data(n_points, true_lengthscale, noise_level)

In [ ]:
# Estimate the correlation lengthscale
lengthscale, dists, correlations, params = compute_lengthscale(x, y, z, epsilon)
print(f"True correlation lengthscale: {true_lengthscale:.4f}")
print(f"Estimated correlation lengthscale: {lengthscale:.4f}")

In [ ]:
# Perform Objective Analysis (OA)
estimated_lengthscale = 0.1  # Initial guess for lc
theta, dists_matrix, C = objective_analysis(x, y, z, estimated_lengthscale, epsilon)

In [ ]:
# Plotting the results
plt.scatter(x, y, c=theta-z, cmap='viridis', label='Gridded Field')
plt.colorbar(label='Theta')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Gridded Field using Objective Analysis (OA)')
plt.legend()
plt.show()

In [ ]:
plt.hist(z-theta)